In [1]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [4]:
from models import *
from utils import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
validation_192_640_pre.csv
training_192_640_pre.csv
training_192_640.csv


In [6]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [7]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,364.362615,362.030694,310.151266,93.752525
1,2011_09_26/2011_09_26_drive_0056_sync/image_02...,2011_09_26/2011_09_26_drive_0056_sync/image_02...,2011_09_26/2011_09_26_drive_0056_sync/image_02...,371.806866,369.427302,314.104631,88.501248
2,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,2011_10_03/2011_10_03_drive_0027_sync/image_03...,370.424992,368.054272,312.885179,94.830438
3,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,364.362615,362.030694,310.151266,93.752525
4,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,2011_09_30/2011_09_30_drive_0028_sync/image_02...,364.362615,362.030694,310.151266,93.752525


In [8]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_26/2011_09_26_drive_0101_sync/image_02...,2011_09_26/2011_09_26_drive_0101_sync/image_02...,2011_09_26/2011_09_26_drive_0101_sync/image_02...,371.806866,369.427302,314.104631,88.501248
1,2011_09_30/2011_09_30_drive_0020_sync/image_03...,2011_09_30/2011_09_30_drive_0020_sync/image_03...,2011_09_30/2011_09_30_drive_0020_sync/image_03...,364.362615,362.030694,310.151266,93.752525
2,2011_09_26/2011_09_26_drive_0022_sync/image_03...,2011_09_26/2011_09_26_drive_0022_sync/image_03...,2011_09_26/2011_09_26_drive_0022_sync/image_03...,371.806866,369.427302,314.104631,88.501248
3,2011_09_26/2011_09_26_drive_0091_sync/image_03...,2011_09_26/2011_09_26_drive_0091_sync/image_03...,2011_09_26/2011_09_26_drive_0091_sync/image_03...,371.806866,369.427302,314.104631,88.501248
4,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,370.424992,368.054272,312.885179,94.830438


In [9]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [10]:
trn_camera.head()

,fx,fy,cx,cy
0,364.362615,362.030694,310.151266,93.752525
1,371.806866,369.427302,314.104631,88.501248
2,370.424992,368.054272,312.885179,94.830438
3,364.362615,362.030694,310.151266,93.752525
4,364.362615,362.030694,310.151266,93.752525


In [11]:
arch = f
bs = 8
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 500

In [12]:
f

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

In [13]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [14]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [15]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [16]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [17]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [19]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        # appr smooth ssim l1
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return losses

In [20]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(3): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [21]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [22]:
TS = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8 ,0.9, 1]; TS
# don't use linspace or your would get into trouble: need to fix this
#TS = [0, 0.15, 0.3, 0.5, 0.75, 0.85, 1]

[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [23]:
expfolder = Path('./experiment/loss/')
expfolder.mkdir(parents=True, exist_ok=True)

In [24]:
for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    
    m = TriDepth(get_resnet(), 1).cuda()
#     set_trainable(m.depth.rn, False)
    l = Loss(scale=1, Tscale=ts).cuda()
    opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)
    print("training model {}".format(ts))
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("saving model")
    save_model(m, thisfolder/"model.M".format(ts))
    print("writting log")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

training model 0
0.5857985615730286	0.5736643671989441	0.012134195305407047	0.5736643671989441	0.0


KeyboardInterrupt: 

for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    m = TriDepth(get_base(), 1).cuda()
    load_model(m, thisfolder/"model.M".format(ts))
    l = Loss(scale=1, Tscale=ts).cuda()
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
m.eval()


In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        

In [ ]:
d2s[0].min(), d2s[0].max()